In [ ]:
#digit recongnizer

In [ ]:
#Import statements
import torch #Import the entire torch library
from torch.utils.data import Dataset, DataLoader #Importing Dataset (Can create custom test and train dataset) and DataLoader (wrapper around Dataset to iterate)
from torchvision.transforms import ToTensor #Transform function allows to convert PIL Images to numpy ndarray, normalize value between [0,1], and HxWxC to CxHxW
import torch.nn as nn #Neural network from pytorch
import pandas as pd
import numpy as np

In [ ]:
#finding best device on which we can run our tensor
if torch.cuda.is_available():
    device=torch.device(type="cuda",index=0)#cuda is for GPU  and idx=0 means from multiple available GPUs assign first one i.e. at 0th idx one
else:
    device=torch.device(type="cpu",index=0)#by defult it will run on CPU

print(torch.cuda.is_available())

print(device)
#we have two ways either we create tensor on GPU or create on CPU and later on we can move it to GPU for computation

True
cuda:0


In [ ]:
class CustomTrainDataset(Dataset):
  def __init__(self, path, transform):#path is for train dataset
    super().__init__()
    self.data=pd.read_csv(path, header="infer").values #header="infer" means we are ignoring header and .values means we are creating into numpy array
    self.length = self.data.shape[0] #shape means (row,col)
    self.transform = transform

  def __len__(self):
      return self.length #returning already calculated length

  def __getitem__(self, idx): # this is to set one input output pair at a time
      flatimage = self.data[idx, 1:].astype(np.uint8) # means we are extracting ith image so it will have length 784
      image = self.transform(np.reshape(flatimage, (28,28,1))) #converring into 2d image like (28*28*1)=784
      label = self.data[idx,0]
      return image, label # so we are returning 2d image and lable for that

class CustomTestDataset(Dataset):# here we do not have any label for testing purpose so from 0th col to 783th col we have image vector
  def __init__(self, path, transform):
    super().__init__()
    self.data = pd.read_csv(path, header="infer").values
    self.length = self.data.shape[0]
    self.transform = transform

  def __len__(self):
    return self.length

  def __getitem__(self, idx):
    flatimage = self.data[idx,:].astype(np.uint8)
    image = self.transform(np.reshape(flatimage, (28,28,1)))
    return image

train_dataset=CustomTrainDataset('/content/drive/MyDrive/Sem 6/DL/Practical-4/data/train.csv', ToTensor())
test_dataset=CustomTestDataset('/content/drive/MyDrive/Sem 6/DL/Practical-4/data/test.csv', ToTensor())

batch_size=64


# now we have to wrap dataloader around our dataset so that it can divide dataset in to batch size of 64
train_dl=DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)
#here if we do shuffle then order of output will be distrubed but in submission we need in order so we are not applying shuffle
test_dl=DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
)
#this class inherits nn.Module
class DigitRecognizer(nn.Module):
  #we are creating various layers we are not joining those  hence we can write lines of init method in any manner
  def __init__(self): #dendor metod
    super().__init__()
    self.relu = nn.ReLU()#defining activation function and common for all layers

    # first of all input will be 1x28x28 so channel will be 1 and we want to produce 8 channels.
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=1, padding=0)
    self.bn1 = nn.BatchNorm2d(8) #8 -> because in output we are getting 8 channels so we need ot pass for 8 channges for normalisation
    self.mp1 = nn.MaxPool2d(kernel_size=(2,2), stride=2, padding=0)

    #here input channel is 8 because batchnorm.. and maxpooling not change to no of feature maps
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=1, padding=0)
    self.bn2 = nn.BatchNorm2d(16)

    self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3,3), stride=1, padding=0)
    self.bn3 = nn.BatchNorm2d(32)

    self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3), stride=1, padding=0)
    self.bn4 = nn.BatchNorm2d(64)

    self.flatten = nn.Flatten()

    self.lin1 = nn.Linear(in_features=3136, out_features=10)#3136 is noting but 7*7*64
    self.bn5 = nn.BatchNorm1d(num_features=10)

  def forward(self, x):
    # output=((w-f+2p)/s)+1
    #so for first layer only we are applying maxpooling.
    # so initialy x is 28x28
    x = self.conv1(x) #now after applying stride and padding it will be 26x26
    x = self.bn1(x)
    x = self.relu(x)
    x = self.mp1(x) #now it will convert into 30x30

    x = self.conv2(x) #now it will convert into 11x11
    x = self.bn2(x)
    x = self.relu(x)

    x = self.conv3(x)#now it will be 9x9
    x = self.bn3(x)
    x = self.relu(x)

    x = self.conv4(x)#now it will be 7x7 and 64 feature maps so
    x = self.bn4(x)
    x = self.relu(x)

    x = self.flatten(x) #now after flatten it will be 7*7*64 means 3136

    x = self.lin1(x) #and we are applying the linear conv so so output will be 10 features for each 64 images
    output = self.bn5(x)

    return output

def train_one_epoch(dataloader, model,loss_fn, optimizer):
    model.train()
    track_loss=0
    num_correct=0
    for i, (imgs, labels) in enumerate(dataloader):
        imgs=imgs.to(device)
        labels=labels.to(device)
        pred=model(imgs)#here it will be 64x1x28x28

        loss=loss_fn(pred,labels)
        track_loss+=loss.item()
        num_correct+=(torch.argmax(pred,dim=1)==labels).type(torch.float).sum().item()

        running_loss=round(track_loss/(i+(imgs.shape[0]/batch_size)),2)
        running_acc=round((num_correct/((i*batch_size+imgs.shape[0])))*100,2)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i%100==0:
            print("Batch:", i+1, "/",len(dataloader), "Running Loss:",running_loss, "Running Accuracy:",running_acc)

    epoch_loss=running_loss
    epoch_acc=running_acc
    return epoch_loss, epoch_acc

def eval(dataloader, model,loss_fn, path):
    model.eval()
    data=pd.read_csv(path) #it have image_id and predicted class
    with torch.no_grad():
        for i, imgs in enumerate(dataloader):#here i is batch size
            imgs=imgs.to(device)
            pred=model(imgs) #so here output will be 64x10

            pred=torch.argmax(pred,dim=1).type(torch.int).cpu()#finding maximum pro idx for all 64 images
            data.iloc[i*batch_size:i*batch_size+batch_size ,1]=pred.numpy() #now let i=0 so from [0:64] it will put predicted class using converting tensor to numpy like tensor.numpy()

    data.to_csv('submission.csv', index=False)
    data.head()



model=DigitRecognizer()
model=model.to(device)
loss_fn=nn.CrossEntropyLoss()
lr=0.001
optimizer=torch.optim.Adam(params=model.parameters(), lr=lr)
n_epochs=3

for i in range(n_epochs):
    print("Epoch No:",i+1)
    train_epoch_loss, train_epoch_acc=train_one_epoch(train_dl,model,loss_fn,optimizer)
    print("Training:", "Epoch Loss:", train_epoch_loss, "Epoch Accuracy:", train_epoch_acc)
    print("--------------------------------------------------")

eval(test_dl, model,loss_fn, '/content/drive/MyDrive/Sem 6/DL/Practical-4/data/sample_submission.csv')

Epoch No: 1
Batch: 1 / 657 Running Loss: 2.73 Running Accuracy: 3.12
Batch: 101 / 657 Running Loss: 0.68 Running Accuracy: 89.98
Batch: 201 / 657 Running Loss: 0.56 Running Accuracy: 93.06
Batch: 301 / 657 Running Loss: 0.49 Running Accuracy: 94.41
Batch: 401 / 657 Running Loss: 0.44 Running Accuracy: 95.26
Batch: 501 / 657 Running Loss: 0.41 Running Accuracy: 95.79
Batch: 601 / 657 Running Loss: 0.38 Running Accuracy: 96.22
Training: Epoch Loss: 0.36 Epoch Accuracy: 96.39
--------------------------------------------------
Epoch No: 2
Batch: 1 / 657 Running Loss: 0.15 Running Accuracy: 98.44
Batch: 101 / 657 Running Loss: 0.19 Running Accuracy: 98.48
Batch: 201 / 657 Running Loss: 0.18 Running Accuracy: 98.57
Batch: 301 / 657 Running Loss: 0.17 Running Accuracy: 98.58
Batch: 401 / 657 Running Loss: 0.17 Running Accuracy: 98.61
Batch: 501 / 657 Running Loss: 0.16 Running Accuracy: 98.56
Batch: 601 / 657 Running Loss: 0.16 Running Accuracy: 98.6
Training: Epoch Loss: 0.15 Epoch Accuracy: